# Optimization - fine tuning

[1. Optimize LSH](#1-optimize-lsh)

[2. Optimize Hypercube](#2-optimize-hypercube)

[3. Optimize GNNS](#3-optimize-gnns)

[4. Optimize MRNG]()

# Import libraries

In [1]:
import optuna
from optuna.visualization import plot_pareto_front, plot_optimization_history, plot_slice

import pandas

from params import lsh_test, hypercube_test, gnn_test, mrng_test

In [2]:
input_path = b'../MNIST/input.dat'
query_path = b'../MNIST/query.dat'

n = 60000

# 1. Optimize LSH

To skip logs, click [here](#visualize-lsh-study-results).

In [5]:
def objective_lsh(trial):
    param_dict = {'k': trial.suggest_int('k', 2, 10),
                  'L': trial.suggest_int('L', 2, 10),
                  'table_size': trial.suggest_categorical('table_size', [int(n/16), int(n/8), int(n/4)]),
                  'window_size': trial.suggest_int('window_size', 1000, 5000)
                 }
    
    print("Trial parameters:", param_dict)

    average_time, true_time, maf, min_neighbors = lsh_test(input_path, query_path, queries_num=10,
                                                           **param_dict, query_trick=True, N=50)

    # trial should return at least 50 neighbors to be used in GNNS
    c0 = - min_neighbors.value + 50
    c1 = - average_time.value + 0.01
    trial.set_user_attr('constraint', (c0, c1))

    return maf.value, average_time.value

def constraints(trial):
    return trial.user_attrs['constraint']

In [6]:
%%time
sampler = optuna.integration.BoTorchSampler(constraints_func=constraints, n_startup_trials=10)
lsh_study = optuna.create_study(study_name='lsh', directions=['minimize', 'minimize'], sampler=sampler)
lsh_study.optimize(objective_lsh, n_trials=50)
print("-------------------- Best trials --------------------")

trials = sorted(lsh_study.best_trials, key=lambda x: x.values)
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}, Constraints = {}".format(trial.values, trial.user_attrs["constraint"]))
    print(" Params = {}".format(trial.params))

<timed exec>:1: ExperimentalWarning: BoTorchSampler is experimental (supported from v2.4.0). The interface can change in the future.
[I 2023-11-25 13:31:12,611] A new study created in memory with name: lsh


Trial parameters: {'k': 10, 'L': 8, 'table_size': 15000, 'window_size': 2003}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-25 13:31:47,486] Trial 0 finished with values: [1.1772753554968618, 0.0022889000000000004] and parameters: {'k': 10, 'L': 8, 'table_size': 15000, 'window_size': 2003}. 


Trial parameters: {'k': 6, 'L': 4, 'table_size': 3750, 'window_size': 2844}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-25 13:32:00,370] Trial 1 finished with values: [1.0323896418551815, 0.0247823] and parameters: {'k': 6, 'L': 4, 'table_size': 3750, 'window_size': 2844}. 


...

Trial parameters: {'k': 10, 'L': 8, 'table_size': 15000, 'window_size': 3738}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-25 14:05:35,747] Trial 48 finished with values: [1.0213835094142336, 0.08483169999999998] and parameters: {'k': 10, 'L': 8, 'table_size': 15000, 'window_size': 3738}. 
/home/elvrach/.local/lib/python3.10/site-packages/optuna/integration/botorch.py:843: ExperimentalWarning: qehvi_candidates_func is experimental (supported from v2.4.0). The interface can change in the future.
  candidates = self._candidates_func(


Trial parameters: {'k': 10, 'L': 8, 'table_size': 15000, 'window_size': 4244}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-25 14:06:54,408] Trial 49 finished with values: [1.0021869427020387, 0.11239860000000002] and parameters: {'k': 10, 'L': 8, 'table_size': 15000, 'window_size': 4244}. 


-------------------- Best trials --------------------
Trial no. 4
 Values = [0.8612792049187826, 0.6233115], Constraints = (0, -0.6133115)
 Params = {'k': 2, 'L': 10, 'table_size': 15000, 'window_size': 4782}
Trial no. 30
 Values = [0.9051289104677074, 0.29210859999999994], Constraints = (0, -0.28210859999999993)
 Params = {'k': 2, 'L': 4, 'table_size': 7500, 'window_size': 3841}
Trial no. 9
 Values = [0.9078231289864939, 0.24525959999999997], Constraints = (0, -0.23525959999999996)
 Params = {'k': 4, 'L': 10, 'table_size': 3750, 'window_size': 3355}
Trial no. 46
 Values = [0.9492472634245511, 0.0918829], Constraints = (0, -0.08188290000000001)
 Params = {'k': 9, 'L': 8, 'table_size': 15000, 'window_size': 4217}
Trial no. 23
 Values = [0.9709495420790043, 0.08566159999999999], Constraints = (0, -0.0756616)
 Params = {'k': 9, 'L': 6, 'table_size': 15000, 'window_size': 4050}
Trial no. 10
 Values = [0.9716113798142378, 0.043435699999999994], Constraints = (0, -0.03343569999999999)
 Param

## Visualize LSH study results

In [13]:
plot_pareto_front(lsh_study, target_names=['maf', 'average_time'])

In [14]:
plot_optimization_history(lsh_study, target = lambda t: t.values[0], target_name = 'maf')

In [15]:
plot_optimization_history(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')

In [16]:
plot_slice(lsh_study, target = lambda t: t.values[0], target_name = 'maf')

In [17]:
plot_slice(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')

# 2. Optimize Hypercube

To skip logs, click [here](#visualize-hypercube-study-results).

In [5]:
def objective_hypercube(trial):
    param_dict = {'k': trial.suggest_int('k', 2, 30),
                  'M': trial.suggest_int('M', 10, 5000),
                  'probes': trial.suggest_int('probes', 1, 1000)
                 }
    
    average_time, true_time, maf = hypercube_test(input_path, query_path, queries_num=10, **param_dict, N=1)

    # penalize model if slower than brute force
    if average_time.value > 0.01:
        return 1000, 1000

    return maf.value, average_time.value

In [6]:
%%time
hypercube_study = optuna.create_study(study_name='hypercube', directions=['minimize', 'minimize'])
hypercube_study.optimize(objective_hypercube, n_trials=50)
print("-----------------------------------------------------")

trials = sorted(hypercube_study.best_trials, key=lambda x: x.values)
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}".format(trial.values))
    print(" Params = {}".format(trial.params))

[I 2023-11-24 20:25:42,322] A new study created in memory with name: hypercube


Read MNIST data
Preprocessing...
Preprocessing time: 2.91206
Done
Algorithm: Cube
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-24 20:25:48,062] Trial 0 finished with values: [1.9820858777568677, 0.0054666] and parameters: {'k': 15, 'M': 3181, 'probes': 31}. 


Read MNIST data
Preprocessing...
Preprocessing time: 5.40988
Done
Algorithm: Cube
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-24 20:25:57,011] Trial 1 finished with values: [1.7737250814991719, 0.0702306] and parameters: {'k': 28, 'M': 1684, 'probes': 200}. 


...

Read MNIST data
Preprocessing...
Preprocessing time: 2.87867
Done
Algorithm: Cube
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-24 20:30:17,488] Trial 48 finished with values: [1.5266650041139054, 0.004064] and parameters: {'k': 15, 'M': 2756, 'probes': 230}. 


Read MNIST data
Preprocessing...
Preprocessing time: 3.07504
Done
Algorithm: Cube
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9


[I 2023-11-24 20:30:23,393] Trial 49 finished with values: [1.4115338545030527, 0.010862] and parameters: {'k': 16, 'M': 1699, 'probes': 847}. 


-----------------------------------------------------
Trial no. 17
 Values = [1.2138728669424221, 0.09235270000000001]
 Params = {'k': 26, 'M': 4453, 'probes': 980}
Trial no. 24
 Values = [1.3877293557617412, 0.001493]
 Params = {'k': 9, 'M': 2943, 'probes': 740}
Trial no. 44
 Values = [1.621818005658229, 0.0011896]
 Params = {'k': 8, 'M': 4351, 'probes': 679}
Trial no. 19
 Values = [1.7008422594541597, 0.0011613]
 Params = {'k': 2, 'M': 24, 'probes': 804}
Trial no. 14
 Values = [1.780159117933763, 0.0009214]
 Params = {'k': 4, 'M': 1966, 'probes': 570}
Trial no. 22
 Values = [1.8083326113265408, 0.0006953]
 Params = {'k': 4, 'M': 3099, 'probes': 359}
Trial no. 9
 Values = [1.9332440185941568, 0.0005749]
 Params = {'k': 4, 'M': 66, 'probes': 305}
Trial no. 10
 Values = [2.381787456413473, 0.00044039999999999997]
 Params = {'k': 8, 'M': 2748, 'probes': 124}
Trial no. 42
 Values = [2.633067072700275, 0.0004029]
 Params = {'k': 5, 'M': 4957, 'probes': 140}
CPU times: user 3min 55s, sys: 4

## Visualize Hypercube study results

In [7]:
plot_pareto_front(hypercube_study, target_names=['maf', 'average_time'])

In [8]:
plot_optimization_history(hypercube_study, target = lambda t: t.values[0], target_name = 'maf')

In [9]:
plot_optimization_history(hypercube_study, target = lambda t: t.values[1], target_name = 'average_time')

In [10]:
plot_slice(hypercube_study, target = lambda t: t.values[0], target_name = 'maf')

In [11]:
plot_slice(hypercube_study, target = lambda t: t.values[1], target_name = 'average_time')

# 3. Optimize GNNS

To skip logs, click [here](#visualize-gnns-study-results).

In [15]:
def objective_gnns(trial):
    param_dict = {'k': trial.suggest_int('k', 10, 100),
                  'E': trial.suggest_int('E', 10, 100),
                  'R': trial.suggest_int('R', 1, 10)
                 }
    
    # penalize bad parameter choice
    if param_dict['E'] > param_dict['k']:
        return 1000, 1000

    average_time, true_time, maf, min_neighbors = gnn_test(input_path, query_path, queries_num=10, **param_dict, N=1)

    return maf.value, average_time.value

NameError: name 'param_dict' is not defined

## Visualize GNNS study results

In [ ]:
plot_pareto_front(gnns_study, target_names=['maf', 'average_time'])

In [ ]:
plot_optimization_history(gnns_study, target = lambda t: t.values[0], target_name = 'maf')

In [ ]:
plot_optimization_history(gnns_study, target = lambda t: t.values[1], target_name = 'average_time')

In [ ]:
plot_slice(gnns_study, target = lambda t: t.values[0], target_name = 'maf')

In [ ]:
plot_slice(gnns_study, target = lambda t: t.values[1], target_name = 'average_time')